In [1]:
import pandas as pd
import numpy as np
import math as m
import DataHandling

In [256]:
roadId = input('Please write the Road ID: ')

Please write the Road ID: 


In [257]:
roads = DataHandling.readRoads(roadId)
bridges = DataHandling.readBridges(roadId)
widths = DataHandling.readWidths(roadId)
traffic = DataHandling.readTraffic(roadId)
roadnames = DataHandling.readBridges('').drop_duplicates('road', keep = 'first')['road'].tolist()

In [258]:
def formatWidthData(dataframe = widths):
    dataframe = dataframe.drop('width', 1)
    dataframe = dataframe.drop('roadId', 1)
    dataframe = dataframe.sort_values(by=['road','startChainage']).reset_index()
    dataframe = dataframe.drop('index', 1)
    return dataframe

In [283]:
def formatBridgeData(dataframe = bridges):
    dataframenoduplicates = dataframe.drop_duplicates('LRPName', keep = 'first')
    dataframemodified = dataframenoduplicates.ix[:,:10]
    dataframemodified = dataframemodified.drop('km', 1)
    dataframemodified = dataframemodified.drop('structureNr', 1)
    dataframemodified = dataframemodified.drop('type', 1)
    dataframemodified = dataframemodified.drop('roadName', 1)
    dataframemodified = dataframemodified.drop('length', 1)
    dataframemodified = dataframemodified.drop('name', 1)
    dataframemodified = dataframemodified.sort_values(by=['road','chainage']).reset_index()
    dataframemodified = dataframemodified.drop('index', 1)
    return dataframemodified

In [260]:
def formatTrafficData(dataframe = traffic, sideofroad = 'both'):
    dataframe = dataframe.drop('name',1)
    dataframe = dataframe.drop('start_lrp',1)
    dataframe = dataframe.drop('start_offset',1)
    dataframe = dataframe.drop('end_lrp',1)
    dataframe = dataframe.drop('end_offset',1)
    dataframe = dataframe.drop('length',1)
    dataframe = dataframe.reset_index()
    dataframe = dataframe.drop('index',1)
    dataframe.columns = dataframe.columns.str.replace('start_chainage','startChainage')
    dataframe.columns = dataframe.columns.str.replace('end_chainage','endChainage')
    dataframe = dataframe.sort_values(by = ['road','startChainage','linkNo'] ).reset_index()
    dataframe = dataframe.drop('index', 1)
    if sideofroad == 'both':
        dataframe = dataframe.groupby(['road','startChainage'],as_index=False).sum()
    return dataframe

In [261]:
def mergeFiles(trafficData=formatTrafficData(), bridgeData = formatBridgeData(), widthData = formatWidthData(), roadside = 'both'):
    indexesMergedData = widthData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
    indexesMergedData =indexesMergedData.tolist()
    addedNrLanes = bridgeData
    list_a = []
    for i in indexesMergedData:
        list_a.append(widthData['nrLanes'][int(i)-1])
    addedNrLanes['nrLanes']=list_a
    bridgeData = addedNrLanes
    addedTrafficData = bridgeData
    for title in trafficData.ix[:,3:].columns:
        indexesMergedData = trafficData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
        indexesMergedData =indexesMergedData.tolist()
        list_b=[]
        for i in indexesMergedData:
            list_b.append(trafficData[title][int(i)-1])
        addedTrafficData[title]=list_b
    return addedTrafficData

In [262]:
def calculateLaneTrafficDensity(dataframe = mergeFiles()):
    for title in dataframe.ix[:,5:].columns:
        dataframe[title] = dataframe[title]/dataframe['nrLanes']
    return dataframe

In [263]:
#Truck capacity is max capacity of type of trucks in tonnes
def calculateVulnerability(dataframe = calculateLaneTrafficDensity(), truckcapacity = [15,10,5], socialcarcapacity = [45,30,15,6,4,2,2,1,2,1], conditionlikelihood = [0.05,0.10,0.15,0.20]):
    vulbasematrix = dataframe.ix[:,:4]
    vulbasematrix.loc[vulbasematrix.condition == 'A', 'BridgeFailureLikelihood'] = conditionlikelihood[0]
    vulbasematrix.loc[vulbasematrix.condition == 'B', 'BridgeFailureLikelihood'] = conditionlikelihood[1]
    vulbasematrix.loc[vulbasematrix.condition == 'C', 'BridgeFailureLikelihood'] = conditionlikelihood[2]
    vulbasematrix.loc[vulbasematrix.condition == 'D', 'BridgeFailureLikelihood'] = conditionlikelihood[3]
    cargotitles = ['heavyTruck', 'mediumTruck', 'smallTruck']
    socialtitles = ['largeBus','mediumBus','microBus','utility','car','autoRickshaw','motorcycle','bicycle','cycleRickshaw','cart']
    vulbasematrix['TrafficEconomicVulnerability'] = 0
    for i in range(len(cargotitles)):
        vulbasematrix['TrafficEconomicVulnerability'] =  vulbasematrix['TrafficEconomicVulnerability']+dataframe[cargotitles[i]]*truckcapacity[i]
    vulbasematrix['TotalEconomicVulnerability'] = vulbasematrix['TrafficEconomicVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    vulbasematrix['TrafficSocialVulnerability'] = 0
    for i in range(len(socialtitles)):
        vulbasematrix['TrafficSocialVulnerability'] =  vulbasematrix['TrafficSocialVulnerability']+dataframe[socialtitles[i]]*socialcarcapacity[i]
    vulbasematrix['TotalSocialVulnerability'] = vulbasematrix['TrafficSocialVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    return vulbasematrix

In [264]:
def printRoadResults(dataframe = calculateVulnerability(),  roadId = roadId, root = 'vulnerability'):
    dataframe.to_csv(root+roadId+'.csv')


In [284]:
def calculatevulnerabilitysegment(dataframe = calculateVulnerability(), groupLevel=5.0):
    global bridges
    result = dataframe.groupby(['road',np.floor(dataframe.index/groupLevel)], as_index = False)
    bridgereliability = result.apply(lambda x: np.product(1-x['BridgeFailureLikelihood'])).reset_index(drop=True)
    lista = []
    for i in bridgereliability:
        lista.append(i.max())
    result = result.max()
    result['BridgeFailureLikelihood'] = 1-pd.DataFrame(lista)
    result['TotalSocialVulnerability'] = result['TrafficSocialVulnerability']*result['BridgeFailureLikelihood']
    result['TotalEconomicVulnerability'] = result['TrafficEconomicVulnerability']*result['BridgeFailureLikelihood']
    result = result.drop('condition',1)
    result['Latitude'].loc[]
    result['Longitude'].loc[]
    return  result

SyntaxError: invalid syntax (<ipython-input-284-72262cda9307>, line 13)

In [281]:
calculatevulnerabilitysegment()

,road,LRPName,chainage,BridgeFailureLikelihood,TrafficEconomicVulnerability,TotalEconomicVulnerability,TrafficSocialVulnerability,TotalSocialVulnerability
0,N1,LRP010c,10.897,0.226219,2260.00,511.255081,8813.00,1993.668598
1,N1,LRP013b,12.688,0.226219,1570.00,355.163928,7035.00,1591.451105
2,N1,LRP020a,19.760,0.226219,1570.00,355.163928,7035.00,1591.451105
3,N1,LRP023d,23.081,0.226219,1570.00,355.163928,7035.00,1591.451105
4,N1,LRP028b,28.315,0.226219,1570.00,355.163928,7035.00,1591.451105
5,N1,LRP031a,30.908,0.226219,1570.00,355.163928,7035.00,1591.451105
6,N1,LRP033a,32.777,0.226219,1570.00,355.163928,7035.00,1591.451105
7,N1,LRP036a,35.408,0.226219,1570.00,355.163928,7035.00,1591.451105
8,N1,LRP051a,50.558,0.226219,1570.00,355.163928,7035.00,1591.451105
9,N1,LRP083b,82.989,0.226219,1570.00,355.163928,7035.00,1591.451105
